1. 문서 내용 읽기
2. 문서 분할(쪼개기)
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음
        - 문서 길이(input)가 길어서 답변 생성에 시간 소요됨 -> 답변 늦어짐
3. 임베딩 -> 벡터 데이터베이스에 저장 
4. 질문이 있으면, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
## 패키지/라이브러리 설치
# %pip install -qU docx2txt langchain_community langchain-text-splitters langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader


## 1. 문서 내용 읽기고 분할 #################################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 백터 데이터베이스에 저장 ####################################
## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정  
embedding = OpenAIEmbeddings(model="text-embedding-3-large")


In [ ]:
## 2.3 벡터 데이터베이스에 저장
## [방법 1] in memory
# database = Chroma.from_documents(
#     documents=document_list,
#     embedding=embedding,
# )

In [3]:
%pip install langchain-pinecone

  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.12.6
    Uninstalling aiohttp-3.12.6:
      Successfully uninstalled aiohttp-3.12.6
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

In [14]:
len(document_list)

23

In [12]:
from langchain_pinecone import PineconeVectorStore


database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name='law-index',
)

database

In [13]:

## 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ######################
# 3.1. 사용자 질문
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query)

## 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ###################
## 4.1. 프롬프트 작성  
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question : {query}
'''

## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)

## 4.3. LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서를 보냄  
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='전세사기피해자에 대한 금융지원은 전세사기피해자의 주거 안정을 돕기 위한 다양한 방법으로 이루어집니다. 특별히 관련 법률에서는 다음과 같은 지원 사항을 포함하고 있습니다:\n\n1. **필요 자금 융자**: 국가 및 지방자치단체는 전세사기피해자 및 특정 임차인의 주거 안정을 보호하기 위해 필요한 자금을 융자하거나 기타 필요한 지원을 제공할 수 있습니다.\n\n2. **우선변제 미수급자 지원**: 「주택임대차보호법」에 따른 우선변제를 받지 못하여 시급한 지원이 필요한 전세사기피해자에게는 주택도시기금에서 주택의 임대차에 필요한 자금을 융자할 수 있도록 하고 있습니다.\n\n3. **채무 불이행 등록 유예**: 「한국자산관리공사 설립 등에 관한 법률」에 따른 금융회사 등은 전세사기피해자의 보증금이 모두 변제되지 않은 경우, 신용정보 집중관리위원회를 통해 해당 대출의 채무 불이행 및 대위변제 등록을 유예하거나 삭제할 수 있습니다.\n\n이러한 지원 조치는 전세사기피해자가 주거 문제로 인한 재정적 어려움을 겪지 않도록 돕기 위한 법적 장치들입니다. 추가로, 전세사기피해자들은 긴급복지지원법에 의한 긴급지원대상자로 간주되어 추가적인 긴급 지원을 받을 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 342, 'prompt_tokens': 4166, 'total_tokens': 4508, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06',